In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

# 실습 내용 : Faster - RCNN vs. Mask - RCNN 

본 실습 내용은 Object Detection 알고리즘 중, 가장 대표적인 Faster-RCNN과 이를 기반으로 Instance segmentation으로 확장한 Mask-RCNN을 비교해보려고 한다.또한 실습은 PyTorch의 Computer Vision과 관련된 라이브러리들을 활용해 진행한다. 두 알고리즘의 결과물의 차이를 비교해보면서, 이를 어떻게 현실 문제에 적용할 수 있을지에 관한 Insight를 주고 싶은 마음이다.

## Faster-RCNN 실습

### - 필요한 패키지들 불러오기

In [0]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import torch
import numpy as np
import cv2

우선적으로 Faster-RCNN에 관하여 진행해보도록 한다. COCO Dataset에 대해 사전에 훈련된 Pre-trained 모델을 이용해 특정 사진을 Input으로 하고, 모델이 사전에 정의된 범주(Class)가 사진 내에 있다고 추론(Inference)하면 어떠한 부분이 사전에 정의된 클래스를 찾아주는지 확인해보도록 하겠다.

Pre-trained 모델을 torchvision 패키지를 이용해 다운 받도록 한다.

In [0]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

아래의 리스트는 COCO Dataset에서 사전에 정의된 범주(Class)에 관한 리스트이다.

In [0]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

### - 추론(Inference)와 관련한 함수 정의

특정 이미지의 주소와 threshold를 입력해주어, 범주 별로 Threshold 이상의 예측 확률 가진 범주들과 해당 객체들에 대한 B.B의 좌표를 반환해주는 함수이다.

In [0]:
def get_prediction_OD(img_path, threshold, model):
    # 추론하고자 하는 이미지 업로드
    # Pre-trained model에 입력할 수 있는 상태로 변화
    img = Image.open(img_path)
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    
    # Pre-trained model을 이용해 업로드한 이미지에 대한 추론과정 진행
    pred = model([img])
    
    # 사전에 정의된 범주에 대한 예측 확률과 B.B 출력
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    
    return pred_boxes, pred_class

특정 이미지를 불러오고, 해당 이미지 위에 어떠한 범주가 사진 내에 있는지, 어디에 있는지(Bounding Box, B.B), B.B에 대한 범주 이름을 출력해주는 함수이다.

In [0]:
def object_detection_api(img_path, model, threshold=0.5, rect_th=3, text_size=3, text_th=3):
    # 사전에 정의된 범주에 관한 예측확률이 0.5 이상인 범주들과 해당 범주들에 대한 B.B를 가져옴
    boxes, pred_cls = get_prediction_OD(img_path, threshold, model)
    
    # 추론하고자 하는 이미지 불러오기
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # B.B 박스를 이미지 위에 입히고, B.B에 대한 범주를 입력해주는 반복문
    for i in range(len(boxes)):
        cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th)
        cv2.putText(img,pred_cls[i], boxes[i][0], cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
    
    # matplotlib 패키지를 이용해 그림 출력
    plt.figure(figsize=(20,30))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [0]:
img_path = "/content/gdrive/My Drive/2019-DL-GIT/Day_5/hands-on/image/"
os.chdir(img_path)

In [28]:
object_detection_api("HCAI.JPG", model, rect_th=3, text_size=3, text_th=3)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
object_detection_api("")

In [0]:
object_detection_api("")

In [0]:
object_detection_api("")

In [0]:
object_detection_api("")

## Mask-RCNN 실습

### Faster-RCNN에서 사용한 외에 추가적인 패키지 불러오기

random 패키지를 불러온다. 불러오는 이유는 B.B 이외에 사전에 정의된 범주에 대한 Instance segmentation을 진행 해야하기 때문이다. 즉, 범주에 대해 색깔을 입혀주어야 하기에, 객체별로 임의의 색깔을 입히기 위함이다.

In [0]:
import random

COCO Dataset에 대한 Mask-RCNN Pre-trained 모델을 불러온다.

In [0]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

이미지에 대해 색깔 마스크 색깔을 정하는 함수이다.

In [0]:
def random_colour_masks(image):
    # 색깔 정하기
    colours = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180],[250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
    
    # R/G/B 에대해 0으로 초기화
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    
    # 색깔을 정해주기
    r[image == 1], g[image == 1], b[image == 1] = colours[random.randrange(0,10)]
    
    # Segmentation에 결과를 Concatenation 진행
    coloured_mask = np.stack([r, g, b], axis=2)
    return coloured_mask

아래의 함수는 Instance segmentation을 하는 함수이다. 기존의 get_prediction_OD함수에 알고리즘을 통해 찾은 범주에 대해 색깔을 입혀주는(Mask) 부분을 추가한 함수이다.

In [0]:
def get_prediction_IS(img_path, threshold, model):
    # 앞 부분과 동일한 코드
    img = Image.open(img_path)
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    
    pred = model([img])
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
    
    # 찾은 객체에 대한 Mask를 입히는 코드
    masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
    
    # 앞 부분과 동일한 코드
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
    
    masks = masks[:pred_t+1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return masks, pred_boxes, pred_class

특정 이미지를 불러오고, 해당 이미지 위에 어떠한 범주가 사진 내에 있는지, 어디에 있는지(B.B), B.B에 대한 범주 이름과 앞의 함수에 추가적으로 범주에 대해 색깔을 입힌뒤 이미지를 출력해주는 함수이다.

In [0]:
def instance_segmentation_api(img_path, model, threshold=0.5, rect_th=3, text_size=3, text_th=3):
    # get_prediction_IS를 이용해 예측된 범주, B.B, 색깔을 입혀줄 위치(Mask)를 계산
    masks, boxes, pred_cls = get_prediction_IS(img_path, threshold, model)
    
    # 이미지 업로드
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    for i in range(len(masks)):
        rgb_mask = random_colour_masks(masks[i])
        img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
        cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th)
        cv2.putText(img,pred_cls[i], boxes[i][0], cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)

    plt.figure(figsize=(20,30))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [0]:
instance_segmentation_api('')

In [0]:
instance_segmentation_api('')

In [0]:
instance_segmentation_api('')

In [0]:
instance_segmentation_api('')

In [0]:
instance_segmentation_api('')